In [2]:
import util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
year = 2014
tag = 'sg-200'
sqlTxt = '''
select permanent_tournament_id, (b_offset).mean as offset, (b_tee).mean as tee, 
       (b_approach).mean as approach, (b_around).mean as around, 
       (b_putting).mean as putting 
    from stan_sg_tournaments
    where year = %s
      and tag = %s;
'''
t_df = util.pd_from_sql(sqlTxt, [year, tag])
t_df

,permanent_tournament_id,offset,tee,approach,around,putting
0,10,70.953,-1.262,-1.239,-1.025,-1.022
1,11,71.660,-1.258,-1.187,-1.051,-0.996
2,20,72.318,-1.367,-1.111,-1.074,-1.009
3,23,72.310,-1.191,-1.163,-1.034,-1.047
4,27,71.479,-1.138,-1.030,-1.019,-0.984
5,28,72.195,-1.034,-1.149,-1.099,-1.008
6,33,72.682,-1.247,-1.064,-1.024,-1.028
7,60,71.611,-1.047,-1.108,-1.073,-1.035
8,473,72.823,-1.256,-1.265,-1.149,-1.006
9,476,71.548,-1.036,-1.073,-0.994,-0.979


In [53]:
sg_map = {}
for row in t_df.as_matrix():
    sg_map[int(row[0])] = row

In [60]:
year = 2014
tag = 'sg-200'
limit = 200
sqlTxt = '''
select * from (select year, player_id, row_number from rank_view where year = %s  limit %s) r 
    join scores_view s 
    using (player_id, year) order by r.row_number, player_id, year, permanent_tournament_id, round;
'''
p_df = util.pd_from_sql(sqlTxt, [year, limit])
p_df = p_df[(pd.notnull(p_df['sg_approach'])) & (pd.notnull(p_df['sg_around']))]


In [67]:
player_scores = p_df.as_matrix()
player_id = [row[0] for row in player_scores ]
actuals = [row[6] for row in player_scores]
def compute_score(row):
    sg = sg_map[int(row[4])]
    return sg[1] + row[7]*sg[2] + row[8]*sg[3] + row[9]*sg[4] + row[10]*sg[5]

estimated = [compute_score(row) for row in player_scores]
comp_df = pd.DataFrame({'player_id': player_id, 'actuals':actuals, 'estimated': estimated})


In [70]:
total = 0
for row in comp_df.as_matrix():
    total += row[0]-row[1]
total/len(comp_df)

0.0063120265123545417

In [22]:
t_df[t_df['permanent_tournament_id'] == 10].as_matrix()[0]

array([ 10.   ,  70.953,  -1.262,  -1.239,  -1.025,  -1.022])